In [13]:
# from qdrant_client import QdrantClient

# qdrant_client = QdrantClient(
#     url="https://bacf7390-f1c5-4b61-a9f2-4443989f3295.us-east4-0.gcp.cloud.qdrant.io:6333",
#     api_key="GACA7KF9JsAA2a0kE-GBAAMdQlPiZxQ73JX42fsEogufcn8A2aQi6w",
# )

# print(qdrant_client.get_collections())

In [14]:
from llama_index.core import (
    SimpleDirectoryReader,
    VectorStoreIndex,
    StorageContext,
    Settings,
    get_response_synthesizer)
from llama_index.core.query_engine import RetrieverQueryEngine, TransformQueryEngine
from llama_index.core.node_parser import SentenceSplitter
from llama_index.core.schema import TextNode, MetadataMode
from llama_index.vector_stores.qdrant import QdrantVectorStore
from llama_index.embeddings.ollama import OllamaEmbedding
from llama_index.llms.ollama import Ollama
from llama_index.core.retrievers import VectorIndexRetriever
from llama_index.core.indices.query.query_transform import HyDEQueryTransform
import qdrant_client
import logging

In [15]:
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

In [16]:
# load the local data directory and chunk the data for further processing
docs = SimpleDirectoryReader(input_dir="data", required_exts=[".pdf"]).load_data(show_progress=True)
text_parser = SentenceSplitter(chunk_size=512, chunk_overlap=100)

Loading files:   0%|          | 0/1 [00:00<?, ?file/s]

Loading files: 100%|██████████| 1/1 [00:03<00:00,  3.60s/file]


In [17]:
# Create a local Qdrant vector store
logger.info("initializing the vector store related objects")
client = qdrant_client.QdrantClient(
    url="https://bacf7390-f1c5-4b61-a9f2-4443989f3295.us-east4-0.gcp.cloud.qdrant.io:6333/",
    api_key="GACA7KF9JsAA2a0kE-GBAAMdQlPiZxQ73JX42fsEogufcn8A2aQi6w")
vector_store = QdrantVectorStore(client=client, collection_name="research_papers")

INFO:__main__:initializing the vector store related objects
INFO:httpx:HTTP Request: GET https://bacf7390-f1c5-4b61-a9f2-4443989f3295.us-east4-0.gcp.cloud.qdrant.io:6333/collections/research_papers/exists "HTTP/1.1 200 OK"


In [18]:
# local vector embeddings model
logger.info("initializing the OllamaEmbedding")
embed_model = OllamaEmbedding(model_name='mxbai-embed-large', base_url='http://localhost:11434')

logger.info("initializing the global settings")
Settings.embed_model = embed_model
Settings.llm = Ollama(model="llama3", base_url='http://localhost:11434',request_timeout=36000)
Settings.transformations = [text_parser]

text_chunks = []
doc_ids = []
nodes = []

INFO:__main__:initializing the OllamaEmbedding
INFO:__main__:initializing the global settings


In [19]:
logger.info("enumerating docs")
for doc_idx, doc in enumerate(docs):
    curr_text_chunks = text_parser.split_text(doc.text)
    text_chunks.extend(curr_text_chunks)
    doc_ids.extend([doc_idx] * len(curr_text_chunks))

INFO:__main__:enumerating docs


In [20]:
logger.info("enumerating text_chunks")
for idx, text_chunk in enumerate(text_chunks):
    node = TextNode(text=text_chunk)
    src_doc = docs[doc_ids[idx]]
    node.metadata = src_doc.metadata
    nodes.append(node)

INFO:__main__:enumerating text_chunks


In [21]:
logger.info("enumerating nodes")
for node in nodes:
    node_embedding = embed_model.get_text_embedding(
        node.get_content(metadata_mode=MetadataMode.ALL)
    )
    node.embedding = node_embedding

INFO:__main__:enumerating nodes


In [ ]:
logger.info("initializing the storage context")
storage_context = StorageContext.from_defaults(vector_store=vector_store)
logger.info("indexing the nodes in VectorStoreIndex")
index = VectorStoreIndex(
    nodes=nodes,
    storage_context=storage_context,
    transformations=Settings.transformations,
)

INFO:__main__:initializing the storage context
INFO:__main__:indexing the nodes in VectorStoreIndex
INFO:httpx:HTTP Request: GET https://bacf7390-f1c5-4b61-a9f2-4443989f3295.us-east4-0.gcp.cloud.qdrant.io:6333/collections/research_papers/exists "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: GET https://bacf7390-f1c5-4b61-a9f2-4443989f3295.us-east4-0.gcp.cloud.qdrant.io:6333/collections/research_papers "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: PUT https://bacf7390-f1c5-4b61-a9f2-4443989f3295.us-east4-0.gcp.cloud.qdrant.io:6333/collections/research_papers/points?wait=true "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: PUT https://bacf7390-f1c5-4b61-a9f2-4443989f3295.us-east4-0.gcp.cloud.qdrant.io:6333/collections/research_papers/points?wait=true "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: PUT https://bacf7390-f1c5-4b61-a9f2-4443989f3295.us-east4-0.gcp.cloud.qdrant.io:6333/collections/research_papers/points?wait=true "HTTP/1.1 200 OK"


In [ ]:
logger.info("initializing the VectorIndexRetriever with top_k as 2")
vector_retriever = VectorIndexRetriever(index=index, similarity_top_k=5)
response_synthesizer = get_response_synthesizer()
logger.info("creating the RetrieverQueryEngine instance")
vector_query_engine = RetrieverQueryEngine(
    retriever=vector_retriever,
    response_synthesizer=response_synthesizer,
)
logger.info("creating the HyDEQueryTransform instance")
hyde = HyDEQueryTransform(include_original=True)
hyde_query_engine = TransformQueryEngine(vector_query_engine, hyde)

INFO:__main__:initializing the VectorIndexRetriever with top_k as 2
INFO:__main__:creating the RetrieverQueryEngine instance
INFO:__main__:creating the HyDEQueryTransform instance


In [ ]:
logger.info("retrieving the response to the query")
query = "what are the topics of CRYPTOGRAPHY AND NETWORK SECURITY in 3rd year and 2nd semister syllabus.get me an overview on every topic."

response = hyde_query_engine.query(str_or_query_bundle=query)

INFO:__main__:retrieving the response to the query


INFO:httpx:HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://bacf7390-f1c5-4b61-a9f2-4443989f3295.us-east4-0.gcp.cloud.qdrant.io:6333/collections/research_papers/points/search "HTTP/1.1 200 OK"


KeyboardInterrupt: 

In [ ]:
print(f"Query : {query}")
print(f"Response : {response}")

In [ ]:
client.close()